<a href="https://colab.research.google.com/github/Tallperson03/koGPT2-nextPhrase_Recommend/blob/main/koGPT2_nextPhrase_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch

In [45]:
!pip install konlpy

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from konlpy.tag import Okt

# KoGPT2 모델과 토크나이저 불러오기
okt = Okt() # 형태소 분석기 객체 생성
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
import json

# JSON 파일 불러오기
with open('final_sentences.json', 'r', encoding='utf-8') as f:
    sentence_data = json.load(f)

# 학습용 텍스트 파일 생성
with open('train.txt', 'w', encoding='utf-8') as f:
    # 모든 카테고리를 순회하며 문장들을 파일에 쓰기
    for category in sentence_data:
        for sentence in category['value']:
            f.write(sentence + '\n')

print("train.txt 파일 생성이 완료되었습니다.")

train.txt 파일 생성이 완료되었습니다.


In [5]:
!pip install datasets -q

In [49]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# 파인튜닝을 위한 데이터셋 및 데이터 콜레이터 준비
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # Masked Language Model이 아니므로 False
)

# 학습을 위한 설정 (TrainingArguments)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/models/kogpt2-finetuned-final", # 학습 결과물이 저장될 폴더
    overwrite_output_dir=True,
    num_train_epochs=3, # 전체 데이터를 3번 반복 학습
    per_device_train_batch_size=8,
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    report_to="none",
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# 파인튜닝 시작
print("파인튜닝을 시작합니다...")
trainer.train()
print("파인튜닝 완료!")

# 파인튜닝된 모델과 토크나이저 저장
trainer.save_model("/content/drive/MyDrive/models/kogpt2-finetuned-final")
tokenizer.save_pretrained("/content/drive/MyDrive/models/kogpt2-finetuned-final")

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


파인튜닝을 시작합니다...


Step,Training Loss


파인튜닝 완료!


('/content/drive/MyDrive/models/kogpt2-finetuned-final/tokenizer_config.json',
 '/content/drive/MyDrive/models/kogpt2-finetuned-final/special_tokens_map.json',
 '/content/drive/MyDrive/models/kogpt2-finetuned-final/tokenizer.json')

In [23]:
import json

# final_words.json 파일 로드
with open('final_words.json', 'r', encoding='utf-8') as f:
    priority_data = json.load(f)

# 모든 품사의 단어를 하나의 set으로 통합
priority_words_set = set()
for category in priority_data['pos_categories'].values():
    priority_words_set.update(category['words'])

print(f"총 {len(priority_words_set)}개의 우선순위 단어를 불러왔습니다.")

총 272개의 우선순위 단어를 불러왔습니다.


In [44]:
def predict_with_priority(text, num_predictions=5):
    """
    (개선됨) 생성된 후보들을 '우선순위 단어 목록'에 따라 재정렬하여 반환합니다.
    """
    # 1. 모델을 통해 충분한 수의 후보군 생성 (기존과 동일)
    input_ids = tokenizer.encode(text, return_tensors='pt').to(model.device)
    input_length = len(input_ids[0])

    beam_outputs = model.generate(
        input_ids,
        max_length=input_length + 10,
        num_beams=num_predictions * 5,  # 더 다양한 후보를 얻기 위해 beam 개수 늘리기
        no_repeat_ngram_size=2,
        num_return_sequences=num_predictions * 5,
        early_stopping=True
    )

    # 2. 다양성 필터를 거친 모든 후보를 일단 수집
    all_predictions = []
    used_keys = set()
    for beam_output in beam_outputs:
        # ... (기존 후처리 로직과 동일) ...
        generated_sequence = tokenizer.decode(beam_output[input_length:], skip_special_tokens=True).strip()
        if generated_sequence:
            first_phrase = generated_sequence.split(' ')[0]
            if '.' in first_phrase: first_phrase = first_phrase.split('.')[0] + '.'
            elif '?' in first_phrase: first_phrase = first_phrase.split('?')[0] + '?'
            elif '!' in first_phrase: first_phrase = first_phrase.split('!')[0] + '!'

            nouns = okt.nouns(first_phrase)
            if not nouns:
                # 명사가 없는 경우 어간 추출
                tagged = okt.pos(first_phrase, stem=True)
                core_key = None
                for word, pos in tagged:
                    if pos in ['Verb', 'Adjective']:
                        core_key = word; break
                if not core_key: core_key = first_phrase
            else:
                core_key = nouns[0]

            if core_key not in used_keys:
                used_keys.add(core_key)
                all_predictions.append(first_phrase.replace('\n', ''))

    # 3. ✨ 우선순위 단어에 따라 재정렬하는 로직 ✨
    priority_results = []
    other_results = []

    for phrase in all_predictions:
        # 어절의 어간/명사가 우선순위 목록에 있는지 확인
        # (정확도를 위해 원형으로 변환 후 비교)
        key_candidate = None
        nouns = okt.nouns(phrase)
        if nouns:
            key_candidate = nouns[0]
        else:
            tagged = okt.pos(phrase, stem=True)
            for word, pos in tagged:
                if pos in ['Verb', 'Adjective', 'Noun']: # 명사도 한번 더 체크
                    key_candidate = word
                    break

        if key_candidate in priority_words_set:
            priority_results.append(phrase)
        else:
            other_results.append(phrase)

    # 우선순위 결과를 먼저, 그 다음 나머지 결과를 붙여서 최종 목록 생성
    final_sorted_list = priority_results + other_results

    return final_sorted_list[:num_predictions]

# --- 수정한 함수로 테스트 ---
input_text = "두려워하지"
predicted_phrases = predict_with_priority(input_text)
print(f"입력: '{input_text}'")
print(f"추천 어절: {predicted_phrases}")

input_text2 = "겁나"
predicted_phrases2 = predict_with_priority(input_text2)
print(f"입력: '{input_text2}'")
print(f"추천 어절: {predicted_phrases2}")

입력: '두려워하지'
추천 어절: ['마세요.']
입력: '겁나'
추천 어절: ['요?', '네요.']
